> **DO NOT EDIT IF INSIDE `caobd_s19` folder** 

# Assignment 3

**Handin in Peergrade**: *Wednesday*, November 13, 2019, 23:59<br>
**Peergrading deadline**: *Sunday*, November 17, 2019, 23:59<br>
**Peergrading feedback deadline**: *Tuesday*, November 19, 2019, 23:59

[**Feedback**](http://ulfaslak.com/vent)

## Problems

### Week 7

>**Ex. 7.2.1**: Compute the sentiment score of each character's page and produce three histograms of sentiment scores, one for each faction. You can use the text strings you generated in the previous exercise. We will cheat a bit and use a library that does the scoring for us. Install `afinn` using `conda` or `pip` and extract the sentiment with that module. There's an example of how to use it on the library's [PyPi repository](https://pypi.python.org/pypi/afinn).

In [7]:
import re
import nltk
import os
import numpy as np

#nltk.download('stopwords')

# Get a list of stopwords from nltk
stopwords = nltk.corpus.stopwords.words("english")

#grabs words for character
def get_clean_words(character_filename, faction):
    def _isnum(w):
        try:
            int(w)
            return True
        except ValueError:
            return False
        
    # Load her markup
    with open("./data/%s/%s" % (faction, character_filename)) as fp:
        markup = fp.read()

    # Remove table and external links
    markup_text = re.sub(r'\{\{[\s\S]*?\}\}', '', markup)

    # Remove category links
    markup_text = re.sub(r'\[\[Category.+\]\]', '', markup_text)

    # Set words to lowercase and remove them if they are stop words
    words = [w.lower() for w in re.findall('\w+', markup_text) if w.lower() not in stopwords]

    # Remove numbers
    words = [w for w in words if not _isnum(w)]

    return words

# Example
#get_clean_words("Iron Man.txt", "heroes")

In [2]:
def extract_marvel_text():
    
    faction_text = {'heroes': [], 'villains': [] , 'ambiguous': []}
    
    for faction in ["heroes", "ambiguous", "villains"]:
        faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
   
        for char in faction_chars:
            text_file = char+".txt"
            
            words = get_clean_words(text_file,faction)
            
            for w in words: 
                if w != char and w != 'ref':
                    faction_text[faction].append(w)
               
    return faction_text
        
        
faction_text = extract_marvel_text()

In [219]:
def extract_marvel_text():
    
    faction_text = {'heroes': [], 'villains': [] , 'ambiguous': []}
    
    for faction in ["heroes", "ambiguous", "villains"]:
        faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
   
        for char in faction_chars:
            text_file = char+".txt"
            
            words = get_clean_words(text_file,faction)
            
            for w in words: 
                if w != char and w != 'ref':
                    faction_text[faction].append(w)
               
    return faction_text
        
        
faction_text = extract_marvel_text()

In [3]:
from afinn import Afinn
afinn = Afinn()

hero_text = ' '.join(faction_text['heroes'])
ambig_text = ' '.join(faction_text['ambiguous'])
villain_text = ' '.join(faction_text['villains'])

hero_score = afinn.score(hero_text)
villain_score = afinn.score(villain_text)
ambig_score = afinn.score(ambig_text)

all_scores = [hero_score, villain_score, ambig_score]

In [4]:
print(hero_score)
print(villain_score)
print(ambig_score)

18770.0
-8475.0
-2535.0


>**Ex. 7.4.1**: Use any tool you like (you can do it manually, it's straight forward if you understand the method), to perform a TF-IDF transform on your BoW matrix from Ex. 7.3.1. The result should be a matrix of the same shape as the BoW, but with different values inside.
1. Explain what these values mean.
2. For the top three most written about characters in each class (so 9 in total), print out each of their 10 highest scoring words. Comment on any differences you observe in the type of words being used in different classes.

> The TF value scores a word based on how often it appears within the current corpus (piece of text)
>
> The IDF value in conjunction with the TF value scores the importance of a word to a corpus compared to other corpuses. This often helps to find words that tell alot about a specific corpus since it rates a word highly if it appears alot within the specific text but not in others.

In [5]:
#for stemming words
from nltk.stem.porter import *
from nltk.corpus import words

dictionary = words.words()

#get list of unique words for matrix
def construct_word_list():
    
    stemmer = PorterStemmer()
    wordSet = set([])
    wordList = []
    #special characters to ommit
    string_check= re.compile('[@_!#$%^&*()<>?/\|}{~:]')
    
    for faction in ["heroes", "ambiguous", "villains"]:
        faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
   
        for char in faction_chars:
            text_file = char+".txt"
            
            words = get_clean_words(text_file,faction)
            
            for w in words:
                w = stemmer.stem(w)
                #ommit stopwords, character names and words starting with numbers
                if w != char and w != 'ref' and w not in stopwords and w[0] > 'A':
                
                    #check for special characters
                    if(string_check.search(w) == None): 
                        wordSet.add(w)
                        
    wordSet = sorted(wordSet)
    
    for w in wordSet:
        wordList.append(w)
        
    return wordList, wordSet

In [8]:
#return total list of chars
def get_target_list():

    target_list = []

    for faction in ['heroes', 'villains', 'ambiguous']:
        for char_filename in os.listdir('./data/%s/' % faction):
            char_name = char_filename[:-4]
            target_list.append([char_name, faction])

    target_list.sort()

    return target_list

target_array = np.array(get_target_list())

target_array_char = target_array[:,0] # list of chars
target_array_faction = target_array[:,1] # list of faction

In [9]:
#VERY SLOW RUN TIME
#creates unclean bow matrix
def construct_bow(wordList, wordSet):
    charList = []
    stemmer = PorterStemmer()
    matrix = np.empty(shape=(len(target_array_char), len(wordList)))
    #special characters to ommit
    string_check= re.compile('[@_!#$%^&*()<>?/\|}{~:]')
    
    i = 0
    
    for faction in ["heroes", "ambiguous", "villains"]:
        faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
   
        for char in faction_chars:
            text_file = char+".txt"
            charList.append(char)
            
            row = np.zeros(len(wordList))
            
            words = get_clean_words(text_file,faction)
            
            
            for w in words:
                w = stemmer.stem(w)
                #ommit stopwords, character names, and words starting with non letters
                if w != char and w != 'ref' and w not in stopwords and w[0] >= 'A' :
                    if(string_check.search(w) == None):
                        index = wordList.index(w)
                        row[index] += 1
                    
            matrix[i] = row
            i += 1
            
    return charList, matrix       

In [10]:
#deletes single instance words from matrix
def clean_bow(wordList, matrix):
    delete_ind = []
    sums = np.sum(matrix, axis=0)
    ind = 0
    
    del_indices = []
    
    for i,n in enumerate(sums):
        if n==1:
            del_indices.append(i)
            
    matrix = np.delete(matrix, del_indices, axis=1)
    wordList = np.array(wordList)
    wordList = np.delete(wordList, del_indices)
    
    return matrix, wordList

In [11]:
#creates the tf_idf matrix for a given BOW matrix
def tf_idf(matrix):

    def find_tf(matrix):
        row_sums = matrix.sum(axis=1)
        tf = matrix / row_sums[:, np.newaxis]
        return tf

    def find_idf(matrix):
        num_rows = len(matrix)
        idf = []

        for word in matrix.T:
            f = 0
            for x in word:
                if x > 0:
                    f += 1
            idf.append(num_rows / f)

        return np.array(idf)


    # part 1: normalize rows
    tf = find_tf(matrix)

    # part 2: compute idf of each word
    idf = find_idf(matrix)

    # part 3: compute tf-idf

    matrix_tfidf = np.empty(matrix.shape)
    for i in range(matrix_tfidf.shape[0]):
        for j in range(matrix_tfidf.shape[1]):
            matrix_tfidf[i][j] = tf[i,j] * idf[j]

    return matrix_tfidf, tf, idf

In [17]:
#counts num of words for each character and its index
def char_word_list():
    word_count = []
    faction_ind = []
    stemmer = PorterStemmer()
    i = 0
    
    #special characters to ommit
    string_check= re.compile('[@_!#$%^&*()<>?/\|}{~:]')

    for faction in ["heroes", "ambiguous", "villains"]:
        faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
        faction_ind.append(i)
   
        for char in faction_chars:
            text_file = char+".txt"
            
            count = 0
            words = get_clean_words(text_file,faction)
            
            for w in words:
                w = stemmer.stem(w)
                #ommit stopwords, character names, and words starting with non letters
                if w != char and w != 'ref' and w not in stopwords and w[0] >= 'A' :
                    if(string_check.search(w) == None):
                        count += 1
            word_count.append((count,char, i))
            i += 1
            
    return word_count, faction_ind

In [13]:
#sorts word count of each factions characters
def highest_word_count(word_count, faction_ind):
    hero_ind = faction_ind[0]
    ambig_ind = faction_ind[1]
    villain_ind = faction_ind[2]
    
    hero = word_count[hero_ind : ambig_ind-1]
    ambig = word_count[ambig_ind : villain_ind-1]
    villain = word_count[villain_ind :]
    
    hero.sort(key=lambda tup: tup[0], reverse = True)
    ambig.sort(key=lambda tup: tup[0], reverse = True)
    villain.sort(key=lambda tup: tup[0], reverse = True)
    
    return hero, ambig, villain

In [14]:
#grab index of highest wrank words for given character index
def highest_rank_words(char_ind, matrix_tfidf):
    row = matrix_tfidf[char_ind]
    
    tmp = np.argsort(row)
    tmp = tmp[::-1]

    inds = tmp[:10]
#     print(row[inds])
    
    return inds

In [15]:
#prints the highest scored words for each faction chars in tfidf matrix
def print_highest_score_words(h_word, a_word, v_word, matrix_tfidf, word_list):
    print("HEROS")
    h = h_word[:3]
    for c in h:
        words = []
        ind = highest_rank_words(c[2],matrix_tfidf)
        for i in ind:
            words.append(word_list[i])
        print(c[1],":",words,"\n")
        
    print("AMBIG")
    a = a_word[:3]
    for c in a:
        words = []
        ind = highest_rank_words(c[2],matrix_tfidf)
        for i in ind:
            words.append(word_list[i])
        print(c[1],":",words,"\n")
        
    print("VILLAIN")
    v = v_word[:3]
    for c in v:
        words = []
        ind = highest_rank_words(c[2],matrix_tfidf)
        for i in ind:
            words.append(word_list[i])
        print(c[1],":",words,"\n")  

In [ ]:
#WARNING, SLOW RUN TIME
wordList, wordSet = construct_word_list()
charList, matrix = construct_bow(wordList, wordSet)
clean_matrix, clean_wordList = clean_bow(wordList, matrix)
matrix_tfidf, tf, idf = tf_idf(clean_matrix)

In [18]:
word_count, faction_ind = char_word_list()
h_word, a_word, v_word = highest_word_count(word_count, faction_ind)
print_highest_score_words(h_word, a_word, v_word, matrix_tfidf, clean_wordList)

HEROS
Psylocke : ['new11', 'issue2', 'issue257', 'betsi', 'slaymast', 'kwannon', 'issue455', 'issue77', 'm2000', 'babe'] 

Cyclops (Marvel Comics) : ['uxm511', 'doel', 'gigawatt', 'eyewear', 'uxm504', 'dryad', 'quartz', 'cyclop', 'jeer', 'shiv'] 

Spider-Man : ['saffel', 'saffel21', 'af', 'saffel124', 'maci', 'kindersly4', 'reelzchannel', 'venom2', 'gcd3', 'kempton'] 

AMBIG
Phoenix Force (comics) : ['vel', 'phoenix', 'avx9', 'avx11', 'darkseid', 'avx8', 'avx5', 'stormphoenix', 'reunifi', 'giraud'] 

Emma Frost : ['uxm', 'xmv2', 'xm', 'astrid', 'emmelin', 'nxmv2', 'xx', 'emmafrostfil', 'cw3', 'nxm'] 

Sabretooth (comics) : ['sangr', 'goda', 'schreiber', 'sabretooth', 'liev', 'clara', 'saul', 'birdi', 'creed', 'fuego'] 

VILLAIN
Doctor Octopus : ['tvodo', 'torbert', 'octaviu', 'smugli', 'rosi', 'schenectadi', 'lamaz', 'carlyl', 'octavius', 'ultimatespiderman'] 

Loki (comics) : ['veriti', 'hiddleston', 'ikol', 'loki', 'gunnar', 'sigyn', 'cbrsiege3', 'var', 'helhorn', 'eldr'] 

Vulture (

> There doesn't seem to be specific differences between each faction, but each character seems to have its own unique words. For instance, Psylocke, Cyclops, and Emma Frost have words that reference specific comic issues. Cyclops' words in particular seem to be very specific to him, mentioning eyewear. For Loki we can see some nordic sounding places such as var, helhorn and even his name backwards as ikol. Most of this things may have to do with the preprossesing of the words used in the matrix.

### Week 8

>**Ex. 8.2.1**: Modify the script from Ex. 8.1.1 so that it instead of word counts outputs the number of characters, words and lines in the file. Post as your answer in two seperate cells, (1) the code in the script in a code cell, and (2) the terminal output in a markdown cell with the text indented by one tab.

    from mrjob.job import MRJob

    class MRWordCounter(MRJob):  

        def mapper(self, _, line):
            yield "chars", len(line)

            for words in line.split():
                yield "words", 1

            yield "lines", 1
    	

        def reducer(self, key, values):
            yield key, sum(values)

    if __name__ == '__main__':
        MRWordCounter.run()

    Running step 1 of 1...
    job output is in /tmp/MapReduce2.USER.20191105.143957.822594/output
    Streaming final output from /tmp/MapReduce2.USER.20191105.143957.822594/output...
    "lines"	4
    "words"	21
    "chars"	88

>**Ex. 8.2.3**: Lets go a bit deeper. In this exercise you will implement a MapReduce-MapReduce operation, which computes the same thing as we computed above, but takes as input friend-network data in a slightly more common format: 

>        A B
>        A C
>        A D
>        B C
>        B D
>        B E
>        C D
>        C E
>        D E

>Each line is a "friend-link". The links are undirected and each only occurs once.

>Your job now, is to produce the same output as you did in Ex. 8.2.2, using this input data. To get started faster, use the template below, which shows how to chain together multiple MapReduce steps. Fill out the template and show the output that you get from the terminal when you run it. Clarify whether it corresponds with the output from Ex. 8.2.2.

>*Hint: Try to write the first MapReduce step such that it outputs key-value pairs that correspond to the input data format from Ex. 8.2.2. Then you can reuse your solution to Ex. 8.2.2 in your second MapReduce step.*

>**Ex. 8.2.4**: We can go even further! Let's add a third MapReduce step to count the number of triangles in a network. Again use this input data:

>        A B
>        A C
>        A D
>        B C
>        B D
>        B E
>        C D
>        C E
>        D E

>to validate that your implementation works. It should produce 7 triangles.

>1. Now compute the number of triangles in [this file](http://snap.stanford.edu/data/facebook_combined.txt.gz) which contains 88234 links in an anonymized facebook network. Don't print the whole output, just report the number you get.
>2. Do the same instead using all 2766607 road segments in California as your input. Go to [this site](https://www.cise.ufl.edu/research/sparse/matrices/SNAP/roadNet-CA.html) and download the data in Matrix Market format (`.mtx`). Unzip the file and remove the first 50 lines from it, since that is just markup that we don't need. The file is pretty big so you can expect it to take some time (~4 minutes on my computer). Report the number you get.

>*Hint: Counting triangles is equivalent to counting "common friends". One way to do that is to just count the collective number of common friends that exist in a network. Depending on your implementation you might want to correct your result by a factor 3, since it is likely that you end up counting each triangle three times (one for each point in it).*

>*Nerdy sidenote: Why would anyone want to count triangles??? Well, in network science there is a lot of statistical measures that include the count of triangles in a network. For example, the [clustering coefficient](https://en.wikipedia.org/wiki/Clustering_coefficient), which reveals the proportion of small closed loops in a network, is computed as the number of realized triangles divided by the number of possible triangles.*